In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp2dr1quqa/pubring.gpg' created
gpg: /tmp/tmp2dr1quqa/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
^C
^C


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [0]:
!ls -l

In [6]:
import keras
import datetime
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, UpSampling2D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, Conv2DTranspose
import os
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
import numpy as np
from skimage import io, color
from keras.preprocessing import image
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint

now = datetime.datetime.now()
image_path = "drive/app/input/256/"

def read_image(img_id, dir):
    try:
        img = load_img(dir + "/" + img_id, target_size=(256,256))
        img = img_to_array(img)
        return img
    except:
        return None

def show_image(image):
    plt.imshow(image/255.)
    plt.show()

def convLayer(input, filters, kernel_size,  stride=1):
    return Conv2D(filters, kernel_size, padding="same", activation="relu", strides=stride)(input)

def printOutput(file, output):
    for y in range(256):
        row = ""
        for x in range(256):
            row += str(output[y, x]) + " "
        file.write(row + "\n")
    file.close()

X = []
files = os.listdir(image_path + "/Train/") #[:512]
for image in files:
    img = read_image(image, image_path)
    if not img is None:
        img = np.array(img, dtype=np.float32)
        X.append(img)

print(len(X))
X = np.array(X, dtype=np.float32)
Xtrain = (1.0/255)*X

test = []
files = os.listdir(image_path + "/Test/") #[512:522]
for image in files:
    img = read_image(image, image_path)
    if not img is None:
        img = np.array(img, dtype=np.float32)
        test.append(img)
test = np.array(test, dtype=np.float32)

test = test*(1.0/255)
test = color.rgb2lab(test)
test = test.reshape(test.shape+(1,))
X_val = X[-10:, :, :, 0]
X_val = X_val.reshape(X_val.shape+(1,))
Y_val = X[-10:, :, :, 1:]/128
X = X[:-10]

0


ValueError: ignored

In [0]:
input_shape = (256, 256, 1)

model_input = Input(shape = input_shape)

model_output = convLayer(model_input, 32, (3, 3))
model_output = convLayer(model_output, 32, (3, 3), stride=2)

model_output = convLayer(model_output, 64, (3, 3))
model_output = convLayer(model_output, 64, (3, 3), stride=2)

model_output = convLayer(model_output, 128, (3, 3))
model_output = convLayer(model_output, 128, (3, 3), stride=2)

model_output = convLayer(model_output, 256, (3, 3))
model_output = convLayer(model_output, 256, (3, 3))

model_output = convLayer(model_output, 256, (3, 3))
model_output = convLayer(model_output, 256, (3, 3))

model_output = Conv2DTranspose((2, 2))(model_output)
model_output = convLayer(model_output, 128, (3, 3))

model_output = Conv2DTranspose((2, 2))(model_output)
model_output = convLayer(model_output, 64, (3, 3))

model_output = Conv2DTranspose((2, 2))(model_output)

model_output = Conv2D(2, (3, 3), activation="tanh", padding="same")(model_output)

model = Model(inputs=model_input, outputs=model_output)

In [0]:
# model.load_weights("drive/app/output/my_model_weights2018-08-10 20:36.h5")
# if(not os.path.exists("/floyd/input/model/my_model_weights.h5")):
datagen = ImageDataGenerator(
    shear_range=0.4,
    zoom_range=0.4,
    rotation_range=40,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator()

batch_size = 32
def batch_generator(batch_size):
    for batch in datagen.flow(Xtrain, batch_size = batch_size):
        lab = color.rgb2lab(batch)
        X = lab[:, :, :, 0]
        X = X.reshape(X.shape+(1,))
        Y = lab[:, :, :, 1:] / 128
        yield ([X, Y])
        
def val_batch_generator(batch_size):
    for batch in val_datagen.flow(Xtrain, batch_size = batch_size):
        lab = color.rgb2lab(batch)
        X = lab[:, :, :, 0]
        X = X.reshape(X.shape+(1,))
        Y = lab[:, :, :, 1:] / 128
        yield ([X, Y])

model.summary()

outputDate = now.strftime("%Y-%m-%d %H:%M")
os.chdir("drive/app/output")
os.mkdir(outputDate)
os.chdir(outputDate)
checkpoint = ModelCheckpoint("best.hdf5",
                            monitor="acc",
                            verbose=1,
                            save_best_only=True,
                            mode="max")

tensorboard = TensorBoard(log_dir=".")
model.compile(loss='mean_squared_error',
            optimizer="rmsprop",
            metrics=['accuracy'])

model.fit_generator(batch_generator(batch_size), callbacks=[tensorboard, checkpoint], epochs=1000, steps_per_epoch=8, validation_data=(X_val, Y_val))


try:
    model.save_weights("model_weights.h5")
# else:
#     model.load_weights("/floyd/input/model/my_model_weights.h5")
except:
    print("Could not save")

model.save("model.h5")

In [0]:
inputs = np.concatenate(test[:, :, :, 0], X_val)

# Test model
output = model.predict(inputs)
output = output * 128
print(output.shape)
# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = inputs[i][:,:,0]
    cur[:,:,1:] = output[i]
    io.imsave(str(i) + ".png", color.lab2rgb(cur))